In [1]:
# Translated to .py by Monique A. Stinson
# 31.12.2016
# Adapted to PandasBiogeme by Michel Bierlaire
# Tue Oct 23 16:15:37 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, log

df = pd.read_csv("telephone.dat",'\t')
database = db.Database("telephone",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)


#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_BM	 = Beta('ASC_BM',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,1)
ASC_LF	 = Beta('ASC_LF',0,None,None,0)
ASC_EF	 = Beta('ASC_EF',0,None,None,0)
ASC_MF	 = Beta('ASC_MF',0,None,None,0)
B_FCOST	 = Beta('B_FCOST',0,None,None,0)
B_MCOST	 = Beta('B_MCOST',0,None,None,0)

# Define here arithmetic expressions for names that are not directly
# available from the data

logcostBM  = DefineVariable('logcostBM', log(cost1),database)
logcostSM  = DefineVariable('logcostSM', log(cost2),database)
logcostLF  = DefineVariable('logcostLF', log(cost3),database)
logcostEF  = DefineVariable('logcostEF', log(cost4),database)
logcostMF  = DefineVariable('logcostMF', log(cost5),database)

#Utilities
V_BM = ASC_BM + B_MCOST * logcostBM
V_SM = ASC_SM + B_MCOST * logcostSM
V_LF = ASC_LF + B_FCOST * logcostLF
V_EF = ASC_EF + B_FCOST * logcostEF
V_MF = ASC_MF + B_FCOST * logcostMF

V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

# The choice model is a logit, with availability conditions
logprob = models.loglogit(V,avail,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_Tel_specific"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


         Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_BM  -0.747    0.157   -4.78 1.79e-06         0.155        -4.82   
ASC_EF  -0.092     1.06 -0.0871    0.931             1      -0.0917   
ASC_LF   0.155     0.77   0.201    0.841         0.691        0.224   
ASC_MF   0.479    0.936   0.512    0.609         0.817        0.587   
B_FCOST  -1.71    0.305    -5.6 2.13e-08         0.273        -6.25   
B_MCOST  -2.16    0.242   -8.93        0         0.243         -8.9   

         Rob. p-value  
ASC_BM       1.42e-06  
ASC_EF          0.927  
ASC_LF          0.823  
ASC_MF          0.557  
B_FCOST      4.02e-10  
B_MCOST             0  
Nbr of observations: 434
LL(0) =    -560.250
LL(beta) = -476.608
rho bar square = 0.139
Output file: logit_Tel_specific~00.html
